In [2]:
import pandas as pd

In [2]:
from sklearn.utils import resample

chunksize = 10 ** 5

train_data = pd.DataFrame()
for chunk in pd.read_csv("/Users/maverick/Downloads/BUAN 6341/train_data.csv", chunksize=chunksize):
    train_data = pd.concat([train_data, chunk])
    
train_labels = pd.DataFrame()
for chunk in pd.read_csv("/Users/maverick/Downloads/BUAN 6341/train_labels.csv", chunksize=chunksize):
    train_labels = pd.concat([train_labels, chunk])

# Randomly select 20% of observations from train_labels
sample_labels = resample(train_labels, replace=False, n_samples=int(len(train_labels) * 0.2), random_state=42)

In [4]:
# Merge the sampled labels with train_data
development_sample = train_data.merge(sample_labels[['customer_ID', 'target']], how="inner", on="customer_ID")

# Save the development sample
development_sample.to_csv("/Users/maverick/Downloads/BUAN 6341/development_sample1.csv", index=False)

In [60]:
chunksize = 10 ** 5

development_sample = pd.DataFrame()
for chunk in pd.read_csv("/Users/maverick/Downloads/BUAN 6341/development_sample1.csv", chunksize=chunksize):
    development_sample = pd.concat([development_sample, chunk])

In [5]:
# Summary statistics
print("\nSummary statistics:")
print(development_sample.describe())

# Check for missing values
print("\nMissing values:")
print(development_sample.isnull().sum())


Summary statistics:
                P_2          D_39           B_1           B_2           R_1  \
count  1.097940e+06  1.107069e+06  1.107069e+06  1.106702e+06  1.107069e+06   
mean   6.565960e-01  1.532376e-01  1.236050e-01  6.220934e-01  7.905943e-02   
std    2.449943e-01  2.705313e-01  2.115772e-01  4.009464e-01  2.268095e-01   
min   -4.205811e-01  9.052854e-09 -1.832537e+00  5.485064e-08  1.065542e-08   
25%    4.809022e-01  4.535612e-03  8.823934e-03  1.064625e-01  2.892853e-03   
50%    6.950928e-01  9.056126e-03  3.118192e-02  8.143289e-01  5.778174e-03   
75%    8.651769e-01  2.367280e-01  1.256333e-01  1.002380e+00  8.664453e-03   
max    1.010000e+00  5.362196e+00  1.324059e+00  1.010000e+00  3.006102e+00   

                 S_3          D_41           B_3           D_42          D_43  \
count  901108.000000  1.106702e+06  1.106702e+06  157137.000000  7.757750e+05   
mean        0.224457  5.996630e-02  1.322328e-01       0.185299  1.542663e-01   
std         0.191222  2.

In [7]:
# Data size
print(f"Number of rows: {development_sample.shape[0]}")
print(f"Number of columns: {development_sample.shape[1]}")

# Data types of features
print("\nData types:")
print(development_sample.dtypes)

# Snapshot of data
print("\nData snapshot:")
print(development_sample.head())

Number of rows: 1107069
Number of columns: 191

Data types:
customer_ID     object
S_2             object
P_2            float64
D_39           float64
B_1            float64
                ...   
D_142          float64
D_143          float64
D_144          float64
D_145          float64
target           int64
Length: 191, dtype: object

Data snapshot:
                                         customer_ID         S_2       P_2  \
0  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-03-11  0.374606   
1  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-04-22  0.414269   
2  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-05-12  0.413310   
3  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-06-10  0.328983   
4  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-07-19  0.496989   

       D_39       B_1       B_2       R_1       S_3      D_41       B_3  ...  \
0  0.033519  0.044293  1.008622  0.001470  0.459235  0.002339  0.006168  ...   
1  0.002516  0.

In [61]:
development_sample['S_2'] = pd.to_datetime(development_sample['S_2'])

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt


# Step 4: One-Hot Encoding
features = development_sample.drop([ 'S_2' ], axis=1).columns.to_list()
cat_col = [i for i in features if i.startswith('B_30')or i.startswith('B_38') or i.startswith('D_114') or i.startswith('D_116')or i.startswith('D_117') or i.startswith('D_120') or i.startswith('D_126') or i.startswith('D_63')or i.startswith('D_64')or i.startswith('D_66') or i.startswith('D_68')]
ind_col = ['customer_ID', 'target']
num_col = [col for col in features if col not in cat_col and col not in ind_col]
# Iterate over each categorical column in cat_col
for col in cat_col:
    # Create dummy variables for the current categorical column
    col_dummies = pd.get_dummies(development_sample[col], prefix=col)
    # Concatenate the dummy variables with the DataFrame
    development_sample = pd.concat([development_sample, col_dummies], axis=1)
    # Drop the original categorical column from the DataFrame
    development_sample.drop(col, axis=1, inplace=True)

# Update cat_col to include the new dummy variable columns
cat_col_updated = [col for col in development_sample.columns if col not in num_col]

# Print the updated cat_col list
print(cat_col_updated)

# Remove 'S_2' and 'customer_ID' from the updated cat_col list
cat_col = [col for col in cat_col_updated if col not in ['S_2', 'customer_ID' , 'target']]

# Print the updated cat_col list
print(cat_col)

['customer_ID', 'S_2', 'target', 'D_63_CL', 'D_63_CO', 'D_63_CR', 'D_63_XL', 'D_63_XM', 'D_63_XZ', 'D_64_-1', 'D_64_O', 'D_64_R', 'D_64_U', 'D_66_0.0', 'D_66_1.0', 'D_68_0.0', 'D_68_1.0', 'D_68_2.0', 'D_68_3.0', 'D_68_4.0', 'D_68_5.0', 'D_68_6.0', 'B_30_0.0', 'B_30_1.0', 'B_30_2.0', 'B_38_1.0', 'B_38_2.0', 'B_38_3.0', 'B_38_4.0', 'B_38_5.0', 'B_38_6.0', 'B_38_7.0', 'D_114_0.0', 'D_114_1.0', 'D_116_0.0', 'D_116_1.0', 'D_117_-1.0', 'D_117_1.0', 'D_117_2.0', 'D_117_3.0', 'D_117_4.0', 'D_117_5.0', 'D_117_6.0', 'D_120_0.0', 'D_120_1.0', 'D_126_-1.0', 'D_126_0.0', 'D_126_1.0']


In [66]:
print(development_sample.shape)

(1107069, 225)


In [67]:
import pandas as pd

df_num_agg = development_sample.groupby('customer_ID')[num_col].agg(['mean', 'std', 'min', 'max', 'last'])
df_num_agg.columns = ['_'.join(x) for x in df_num_agg.columns]
print(df_num_agg)
print(df_num_agg.shape)
development_sample = pd.merge(development_sample, df_num_agg, on='customer_ID')
print(development_sample.shape)

                                                    P_2_mean   P_2_std  \
customer_ID                                                              
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...  0.444606  0.065085   
000445609ff2a39d2dd02484899affa5696210a95f6869f...  0.972260  0.010865   
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff...  0.665203  0.028379   
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab7040...  0.970309  0.005252   
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470f...  0.953131  0.031159   
...                                                      ...       ...   
fffe2bc02423407e33a607660caeed076d713d8a5ad3232...  0.513501  0.097925   
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd...  0.859327  0.012425   
ffff9984b999fccb2b6127635ed0736dda94e544e67e026...  0.786838  0.014312   
ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf388145...  0.804454  0.037442   
fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eab...  0.983617  0.012879   

                                     

(1107069, 1110)


In [70]:
# Iterate over each categorical feature
for feature in cat_col:
    # Check if the feature is binary (0/1)
    if development_sample[feature].nunique() == 2:
        # Calculate the response rate (percentage of times the feature equals 1) in the last 6 months
        response_rate_6 = development_sample[feature].rolling(window=window_sizes['response_rate_6'], min_periods=1).mean() * 100
        # Check if the feature has ever been responded (contains at least one 1) in the last 12 months
        ever_response_12 = development_sample[feature].rolling(window=window_sizes['ever_response_12'], min_periods=1).max()
        # Add the aggregated features to the DataFrame
        df_cat_agg[f"{feature}_response_rate_6"] = response_rate_6
        df_cat_agg[f"{feature}_ever_response_12"] = ever_response_12
print(df_cat_agg)
print(df_cat_agg.shape)

                                                    D_66_mean  D_66_max  \
customer_ID                                                               
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f523...        NaN       NaN   
000445609ff2a39d2dd02484899affa5696210a95f6869f...        NaN       NaN   
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff...        NaN       NaN   
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab7040...        NaN       NaN   
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470f...        NaN       NaN   
...                                                       ...       ...   
fffe2bc02423407e33a607660caeed076d713d8a5ad3232...        NaN       NaN   
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd...        NaN       NaN   
ffff9984b999fccb2b6127635ed0736dda94e544e67e026...        NaN       NaN   
ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf388145...        NaN       NaN   
fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eab...        NaN       NaN   

                        

In [71]:
development_sample = pd.merge(development_sample, df_cat_agg, on='customer_ID')
print(development_sample.shape)

(1107069, 1208)


In [72]:
# Save the DataFrame to CSV
development_sample.to_csv("final_data_for_project.csv", index=False)

In [73]:
chunksize = 10 ** 5

dev_sample = pd.DataFrame()
for chunk in pd.read_csv("/Users/maverick/Downloads/BUAN 6341/final_data_for_project.csv", chunksize=chunksize):
    dev_sample = pd.concat([dev_sample, chunk])

In [76]:
dev_sample['S_2'] = pd.to_datetime(dev_sample['S_2'])

# Data types of features
print("\nData types:")
print(dev_sample.dtypes)


Data types:
customer_ID                            object
S_2                            datetime64[ns]
P_2                                   float64
D_39                                  float64
B_1                                   float64
                                    ...      
D_126_-1.0_ever_response_12           float64
D_126_0.0_response_rate_6             float64
D_126_0.0_ever_response_12            float64
D_126_1.0_response_rate_6             float64
D_126_1.0_ever_response_12            float64
Length: 1208, dtype: object


In [79]:
# Step 6: Train/Test Split
from sklearn.model_selection import train_test_split

X = dev_sample.drop(['target','S_2','customer_ID'], axis=1)
Y = dev_sample['target']

# Splitting into 70% training and 30% testing
X_train, X_test_combined, Y_train, y_test_combined = train_test_split(X, Y, test_size=0.3, random_state=42)

# Splitting the combined test set into two equal parts
X_test1, X_test2, Y_test1, Y_test2 = train_test_split(X_test_combined, y_test_combined, test_size=0.5, random_state=42)

In [81]:
print (X_train.shape)
print (Y_train.shape)
print (X_test1.shape)
print (Y_test1.shape)
print (X_test2.shape)
print (Y_test2.shape)

(774948, 1205)
(774948,)
(166060, 1205)
(166060,)
(166061, 1205)
(166061,)


In [82]:
# Step 7-10: Feature Selection
from xgboost import XGBClassifier


# for this step, we don't play with parameters of RF, and just use the
model_for_feature_reduction = XGBClassifier()
model_for_feature_reduction.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [83]:
Feature_Importance = pd.DataFrame(columns = ["Feature", "Feature_Importance"])
Feature_Importance.Feature = X_train.columns
Feature_Importance.Feature_Importance = model_for_feature_reduction.feature_importances_
Feature_Importance.sort_values(by=["Feature_Importance"], inplace=True, ascending=False)
Feature_Importance
Feature_Importance.to_csv("Feature_Importance.csv", index=False)

In [84]:
Feature_Importance

,Feature,Feature_Importance
226,P_2_last,0.140401
236,B_1_last,0.043715
241,B_2_last,0.011351
275,D_44_max,0.010522
246,R_1_last,0.009361
...,...,...
139,D_109,0.000000
140,D_110,0.000000
141,D_111,0.000000
142,B_39,0.000000


In [85]:
params = {
    'n_estimators': 300,
    'learning_rate': 0.5,
    'max_depth': 4,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'scale_pos_weight': 5,
    'random_state': 42  # Set random state for reproducibility
}

# Initialize XGBoost model
model_for_feature_reduction_2 = xgb.XGBClassifier(**params)
# Train the model
model_for_feature_reduction_2.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [86]:
Feature_Importance_2 = pd.DataFrame(columns = ["Feature", "Feature_Importance"])
Feature_Importance_2.Feature = X_train.columns
Feature_Importance_2.Feature_Importance = model_for_feature_reduction_2.feature_importances_
Feature_Importance_2.sort_values(by=["Feature_Importance"], inplace=True, ascending=False)
Feature_Importance_2
Feature_Importance_2.to_csv("Feature_Importance_2.csv", index=False)

In [87]:
Feature_Importance_2

,Feature,Feature_Importance
226,P_2_last,0.101582
224,P_2_min,0.049019
241,B_2_last,0.043612
557,B_23_mean,0.029483
276,D_44_last,0.021654
...,...,...
132,D_106,0.000000
133,D_107,0.000000
134,B_36,0.000000
135,B_37,0.000000


In [89]:
features_to_keep = Feature_Importance[Feature_Importance.Feature_Importance >= 0.005]["Feature"]
features_to_keep

226     P_2_last
236     B_1_last
241     B_2_last
275     D_44_max
246     R_1_last
561    B_23_last
296     R_2_last
262    D_42_mean
Name: Feature, dtype: object

In [90]:
features_to_keep_2 = Feature_Importance_2[Feature_Importance_2.Feature_Importance >= 0.005]["Feature"]
features_to_keep_2

226      P_2_last
224       P_2_min
241      B_2_last
557     B_23_mean
276     D_44_last
236      B_1_last
237      B_2_mean
243       R_1_std
311     D_48_last
267     D_43_mean
929     D_111_min
323       B_7_std
326      B_7_last
401      S_7_last
346      B_9_last
266     D_42_last
261      B_3_last
39           D_56
55           B_19
286     D_45_last
247      S_3_mean
1108     D_66_max
Name: Feature, dtype: object

In [93]:
all_features_to_keep = pd.concat([features_to_keep, features_to_keep_2]).unique()
all_features_to_keep

array(['P_2_last', 'B_1_last', 'B_2_last', 'D_44_max', 'R_1_last',
       'B_23_last', 'R_2_last', 'D_42_mean', 'P_2_min', 'B_23_mean',
       'D_44_last', 'B_2_mean', 'R_1_std', 'D_48_last', 'D_43_mean',
       'D_111_min', 'B_7_std', 'B_7_last', 'S_7_last', 'B_9_last',
       'D_42_last', 'B_3_last', 'D_56', 'B_19', 'D_45_last', 'S_3_mean',
       'D_66_max'], dtype=object)

In [94]:
X_train = X_train[all_features_to_keep]
X_test1 = X_test1[all_features_to_keep]
X_test2 = X_test2[all_features_to_keep]

In [95]:
X_test2

,P_2_last,B_1_last,B_2_last,D_44_max,R_1_last,B_23_last,R_2_last,D_42_mean,P_2_min,B_23_mean,...,B_7_last,S_7_last,B_9_last,D_42_last,B_3_last,D_56,B_19,D_45_last,S_3_mean,D_66_max
905105,0.944889,0.006522,0.813729,0.007334,0.001490,0.003793,0.005450,NaN,0.731393,0.017465,...,0.002318,0.057272,0.000211,NaN,0.002297,0.013266,0.001560,0.181687,0.098454,NaN
232514,0.395721,0.206755,0.205328,0.130624,0.009945,0.100245,0.009267,NaN,0.395721,0.089684,...,0.112766,0.406000,0.373772,NaN,0.180138,NaN,0.003380,0.076016,0.281478,NaN
520289,0.407848,0.026704,0.539601,0.134964,0.000956,0.015782,0.009595,0.022558,0.261217,0.016208,...,0.033183,0.162881,0.331855,0.019468,0.028668,0.653317,0.002509,0.047323,0.147232,NaN
908247,0.066303,1.323635,0.000109,NaN,0.005134,1.197610,1.007439,NaN,-0.020712,0.822233,...,1.247387,0.142377,0.690239,NaN,0.001763,0.097015,0.911675,0.281252,0.294595,NaN
876576,0.692296,0.008489,0.810278,0.007993,0.000676,0.009900,0.006733,NaN,0.692296,0.027126,...,0.013209,0.115667,0.005582,NaN,0.021407,NaN,0.007780,0.146952,0.182291,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051738,0.700425,0.232040,0.005583,0.134289,0.005612,0.211737,1.005927,NaN,0.451747,0.195204,...,0.221383,0.279596,0.618925,NaN,0.521868,0.232324,0.500362,0.096927,0.603322,NaN
390981,0.299608,0.009644,0.810570,0.509578,1.003720,0.390946,1.008630,0.238139,0.140164,0.287967,...,0.425283,0.172965,0.001131,0.224033,0.002898,NaN,0.001839,0.038563,0.166471,NaN
1019489,0.090953,0.083692,0.009631,0.009242,0.000402,0.062427,0.006524,NaN,0.064651,0.051257,...,0.086880,0.226418,0.202196,NaN,0.048573,NaN,0.509463,0.078212,0.208819,NaN
149513,0.874267,0.005580,0.811426,0.009517,0.000154,0.321782,0.002114,NaN,0.740767,0.205339,...,0.419930,NaN,0.002092,NaN,0.001991,NaN,0.005923,0.091765,NaN,0.0


In [96]:
from sklearn.metrics import roc_auc_score

In [97]:
# Step 11: XGBoost Grid Search
Grid_Search_Results = pd.DataFrame(columns = ["Number Trees", "Learning Rate", 'Subsample', '% Features', 'Weight of Default',"AUC Train", "AUC Test 1", "AUC Test 2"])

Counter = 0
for n_trees in [50, 100, 300]:
    for lr in [0.01, 0.1]:
        for subsample in [0.5, 0.8]:
            for colsample in [0.5, 1.0]:
                for weight in [1, 5, 10]:
                    xgb_instance = XGBClassifier(n_estimators= n_trees, learning_rate = lr,subsample=subsample, colsample_bytree=colsample, scale_pos_weight=weight)
                    model = xgb_instance.fit(X_train, Y_train)
                    Grid_Search_Results.loc[Counter,"Number Trees"] = n_trees
                    Grid_Search_Results.loc[Counter,"Learning Rate"] = lr
                    Grid_Search_Results.loc[Counter,"Subsample"] = subsample
                    Grid_Search_Results.loc[Counter,"% Features"] = colsample
                    Grid_Search_Results.loc[Counter,"Weight of Default"] = weight
                    Grid_Search_Results.loc[Counter,"AUC Train"] = roc_auc_score(Y_train, model.predict_proba(X_train)[:,1])
                    Grid_Search_Results.loc[Counter,"AUC Test 1"] = roc_auc_score(Y_test1, model.predict_proba(X_test1)[:,1])
                    Grid_Search_Results.loc[Counter,"AUC Test 2"] = roc_auc_score(Y_test2, model.predict_proba(X_test2)[:,1])

                    Counter = Counter + 1
                    Grid_Search_Results.to_csv("/Users/maverick/Downloads/BUAN 6341/Grid_Search_Results.csv")

In [98]:
print(Grid_Search_Results)

   Number Trees Learning Rate Subsample % Features Weight of Default  \
0            50          0.01       0.5        0.5                 1   
1            50          0.01       0.5        0.5                 5   
2            50          0.01       0.5        0.5                10   
3            50          0.01       0.5        1.0                 1   
4            50          0.01       0.5        1.0                 5   
..          ...           ...       ...        ...               ...   
67          300           0.1       0.8        0.5                 5   
68          300           0.1       0.8        0.5                10   
69          300           0.1       0.8        1.0                 1   
70          300           0.1       0.8        1.0                 5   
71          300           0.1       0.8        1.0                10   

   AUC Train AUC Test 1 AUC Test 2  
0   0.954004   0.953278   0.954243  
1   0.952543   0.951857   0.952767  
2    0.95192   0.951274 

In [151]:
# Step 12: Choose best XGBoost model
best_model_idx = Grid_Search_Results[['AUC Train', 'AUC Test 1', 'AUC Test 2']].mean(axis=1).idxmax()
best_params = Grid_Search_Results.iloc[best_model_idx, :5].to_dict()

In [152]:
best_xgb_instance = xgb.XGBClassifier(
    n_estimators=best_params['Number Trees'],
    learning_rate=best_params['Learning Rate'],
    subsample=best_params['Subsample'],
    colsample_bytree=best_params['% Features'],
    scale_pos_weight=best_params['Weight of Default']
)

# Fit the model on the training data
best_xgb_model = best_xgb_instance.fit(X_train, Y_train)

In [105]:
# Step 13: Data Processing for Neural Network
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy='constant', fill_value=0)
scaler = StandardScaler()

X_train_nn = imputer.fit_transform(X_train)
X_test1_nn = imputer.transform(X_test1)
X_test2_nn = imputer.transform(X_test2)

X_train_nn = scaler.fit_transform(np.clip(X_train_nn, np.percentile(X_train_nn, 1), np.percentile(X_train_nn, 99)))
X_test1_nn = scaler.transform(np.clip(X_test1_nn, np.percentile(X_train_nn, 1), np.percentile(X_train_nn, 99)))
X_test2_nn = scaler.transform(np.clip(X_test2_nn, np.percentile(X_train_nn, 1), np.percentile(X_train_nn, 99)))

In [117]:
# Step 13: Data Processing for Neural Network
outlier = pd.DataFrame(columns=["Column Name", "P1", "P99"])

counter = 0
for feature in all_features_to_keep:
    outlier.loc[counter, "Column Name"] = feature
    outlier.loc[counter, "P1"] = dev_sample[feature].quantile(0.01)
    outlier.loc[counter, "P99"] = dev_sample[feature].quantile(0.99)
    counter += 1

outlier

,Column Name,P1,P99
0,P_2_last,-0.083832,1.006322
1,B_1_last,0.000496,1.054248
2,B_2_last,0.001835,1.009701
3,D_44_max,0.006981,1.255546
4,R_1_last,0.000112,1.502031
5,B_23_last,0.001447,1.045133
6,R_2_last,0.00011,1.008703
7,D_42_mean,0.004561,0.947784
8,P_2_min,-0.129115,0.978856
9,B_23_mean,0.005064,0.902647


In [119]:
# Next we replace outlers with P1 and P99
import numpy as np

for counter in range (outlier.shape[0]):
    X_train[outlier.loc[counter, "Column Name"]] = np.where(X_train[outlier.loc[counter, "Column Name"]] < outlier.loc[counter, "P1"],
                                                       outlier.loc[counter, "P1"], X_train[outlier.loc[counter, "Column Name"]])

    X_train[outlier.loc[counter, "Column Name"]] = np.where(X_train[outlier.loc[counter, "Column Name"]] > outlier.loc[counter, "P99"],
                                                       outlier.loc[counter, "P99"], X_train[outlier.loc[counter, "Column Name"]])


In [122]:
# Next we do the same for test samples. Note we use the same P1/P99 that we got from train sample.
# Test sample represents unseen data, and should not be used in any stage of the model, including data processing.

for counter in range (outlier.shape[0]):
    X_test1[outlier.loc[counter, "Column Name"]] = np.where(X_test1[outlier.loc[counter, "Column Name"]] < outlier.loc[counter, "P1"],
                                                       outlier.loc[counter, "P1"], X_test1[outlier.loc[counter, "Column Name"]])

    X_test2[outlier.loc[counter, "Column Name"]] = np.where(X_test2[outlier.loc[counter, "Column Name"]] < outlier.loc[counter, "P1"],
                                                       outlier.loc[counter, "P1"], X_test2[outlier.loc[counter, "Column Name"]])

    X_test1[outlier.loc[counter, "Column Name"]] = np.where(X_test1[outlier.loc[counter, "Column Name"]] > outlier.loc[counter, "P99"],
                                                       outlier.loc[counter, "P99"], X_test1[outlier.loc[counter, "Column Name"]])

    X_test2[outlier.loc[counter, "Column Name"]] = np.where(X_test2[outlier.loc[counter, "Column Name"]] > outlier.loc[counter, "P99"],
                                                       outlier.loc[counter, "P99"], X_test2[outlier.loc[counter, "Column Name"]])


In [123]:
# Feature Scaling: We will use StandardScaler. There are other scaling options such as Min-Max Scaler.
# No matter which technique to use, again scaling parameters (here mean and STD) should come from the train sample.
# To find scaling parameters, we use a sklearn package.

# get scaling parameters
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)

# scale features
X_train = pd.DataFrame(sc.transform(X_train), columns = X_train.columns)
X_test1 = pd.DataFrame(sc.transform(X_test1), columns = X_test1.columns)
X_test2 = pd.DataFrame(sc.transform(X_test2), columns = X_test2.columns)

In [115]:
# For missing value imputation, we replace all missing values with 0
X_train

,P_2_last,B_1_last,B_2_last,D_44_max,R_1_last,B_23_last,R_2_last,D_42_mean,P_2_min,B_23_mean,...,B_7_last,S_7_last,B_9_last,D_42_last,B_3_last,D_56,B_19,D_45_last,S_3_mean,D_66_max
182769,0.496959,0.287810,0.056929,0.133076,0.001392,0.125750,0.005378,0.000000,0.496959,0.055653,...,0.129357,0.090073,1.618720,0.000000,0.222977,0.145412,0.001598,0.195625,0.184165,0.0
241802,0.859407,0.013125,1.009685,0.009289,0.008317,0.008875,0.005138,0.000000,0.769543,0.021746,...,0.007024,0.064558,0.001715,0.000000,0.003450,0.352785,0.001588,0.137131,0.102559,0.0
769241,0.945290,0.027036,1.000382,0.007802,0.000314,0.013022,0.005344,0.000000,0.515443,0.010340,...,0.025825,0.069300,0.001949,0.000000,0.002920,0.170342,0.001958,0.727309,0.094266,0.0
786784,0.740042,0.146528,0.072188,0.255785,0.005522,0.453014,0.004411,0.000000,0.649869,0.425415,...,0.484120,0.090422,0.076865,0.000000,0.283841,0.058234,0.546189,0.360008,0.206411,0.0
975133,0.660155,0.010987,1.000730,0.009391,0.006295,0.011107,0.003893,0.000000,0.618985,0.086327,...,0.012614,0.015169,0.022663,0.000000,0.054372,0.000000,0.187379,0.403125,0.123456,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,0.656411,0.044336,1.007016,0.131403,0.002799,0.057703,0.008155,0.000000,0.651503,0.043704,...,0.063858,0.075845,0.060060,0.000000,0.025882,0.891379,0.085693,0.538916,0.094249,0.0
259178,0.387004,0.214951,0.024962,0.884363,0.006598,0.427526,0.005532,0.051599,0.308541,0.287140,...,0.428833,0.238701,0.635126,0.045908,0.234551,0.000000,0.663422,0.024923,0.556933,0.0
131932,-0.126758,0.282574,0.059192,0.378495,0.504873,0.568819,1.001571,0.000000,-0.126758,0.521570,...,0.583438,0.506010,0.631523,0.000000,0.605640,0.000000,0.008190,0.514414,0.682694,0.0
671155,0.690684,0.011292,1.000960,0.009987,0.008492,0.011890,0.005936,0.000000,0.537670,0.016471,...,0.010738,0.054497,0.032934,0.000000,0.011278,0.000000,0.004876,0.236419,0.051486,0.0


In [124]:
# For missing value imputation, we replace all missing values with 0
X_train.fillna(0,inplace=True)
X_test1.fillna(0,inplace=True)
X_test2.fillna(0,inplace=True)

In [118]:
all_features_to_keep

array(['P_2_last', 'B_1_last', 'B_2_last', 'D_44_max', 'R_1_last',
       'B_23_last', 'R_2_last', 'D_42_mean', 'P_2_min', 'B_23_mean',
       'D_44_last', 'B_2_mean', 'R_1_std', 'D_48_last', 'D_43_mean',
       'D_111_min', 'B_7_std', 'B_7_last', 'S_7_last', 'B_9_last',
       'D_42_last', 'B_3_last', 'D_56', 'B_19', 'D_45_last', 'S_3_mean',
       'D_66_max'], dtype=object)

In [128]:
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

In [129]:
from sklearn.metrics import roc_auc_score

In [130]:
Grid_Search_Results_nn = pd.DataFrame(columns = ["HL", "Nodes", "Activation Function","Dropout", "Batch Size",
                                              "AUC Train", "AUC Test 1", "AUC Test 2"])

In [135]:
Counter = 0
for n_layers in [2, 4]:
    for n_nodes in [4, 6]:
        for activation in ['relu', 'tanh']:
            for dropout in [0 , 0.5]:
                for batch_size in [100, 10000]:
                    model = Sequential()
                    model.add(Dense(n_nodes, input_dim=X_train_nn.shape[1], activation=activation))
                    model.add(Dropout(dropout))
                    for _ in range(n_layers - 1):
                        model.add(Dense(n_nodes, activation=activation))
                        model.add(Dropout(dropout))
                    model.add(Dense(1, activation='sigmoid'))
                    model.compile(optimizer=Adam(), loss=BinaryCrossentropy())
                    model.fit(X_train_nn, Y_train, batch_size=batch_size, epochs=20, verbose=0)

                    Grid_Search_Results_nn.loc[Counter,"HL"] = n_layers
                    Grid_Search_Results_nn.loc[Counter,"Nodes"] = n_nodes
                    Grid_Search_Results_nn.loc[Counter,"Activation Function"] = activation
                    Grid_Search_Results_nn.loc[Counter,"Dropout"] = dropout
                    Grid_Search_Results_nn.loc[Counter,"Batch Size"] = batch_size
                    Grid_Search_Results_nn.loc[Counter,"AUC Train"] = roc_auc_score(Y_train, model.predict(X_train))
                    Grid_Search_Results_nn.loc[Counter,"AUC Test 1"] = roc_auc_score(Y_test1, model.predict(X_test1))
                    Grid_Search_Results_nn.loc[Counter,"AUC Test 2"] = roc_auc_score(Y_test2, model.predict(X_test2))

                    Counter = Counter + 1
                    Grid_Search_Results_nn.to_csv("/Users/maverick/Downloads/BUAN 6341/Grid_Search_Results_NN.csv")

24218/24218 ━━━━━━━━━━━━━━━━━━━━ 14s 594us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 540us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 536us/step


/Users/maverick/anaconda3/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24218/24218 ━━━━━━━━━━━━━━━━━━━━ 13s 555us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 534us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 543us/step


/Users/maverick/anaconda3/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24218/24218 ━━━━━━━━━━━━━━━━━━━━ 16s 642us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 543us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 537us/step


...

24218/24218 ━━━━━━━━━━━━━━━━━━━━ 14s 588us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 530us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 535us/step


/Users/maverick/anaconda3/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24218/24218 ━━━━━━━━━━━━━━━━━━━━ 16s 645us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 550us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 588us/step


/Users/maverick/anaconda3/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24218/24218 ━━━━━━━━━━━━━━━━━━━━ 15s 632us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 583us/step
5190/5190 ━━━━━━━━━━━━━━━━━━━━ 3s 610us/step


In [146]:
# Step 15: Choose best Neural Network model
best_model_idx = Grid_Search_Results_nn[['AUC Train', 'AUC Test 1', 'AUC Test 2']].mean(axis=1).idxmax()
best_params = Grid_Search_Results_nn.iloc[best_model_idx, :5].to_dict()
best_params

{'HL': 4,
 'Nodes': 6,
 'Activation Function': 'tanh',
 'Dropout': 0,
 'Batch Size': 100}

In [149]:
best_nn_model = Sequential()
best_nn_model.add(Dense(best_params['Nodes'], input_dim=X_train_nn.shape[1], activation=best_params['Activation Function']))
best_nn_model.add(Dropout(best_params['Dropout']))
for _ in range(best_params['HL'] - 1):
    best_nn_model.add(Dense(best_params['Nodes'], activation=best_params['Activation Function']))
    best_nn_model.add(Dropout(best_params['Dropout']))
best_nn_model.add(Dense(1, activation='sigmoid'))
best_nn_model.compile(optimizer=Adam(), loss=BinaryCrossentropy())
best_nn_model.fit(X_train_nn, Y_train, batch_size=best_params['Batch Size'], epochs=20, verbose=0)


In [154]:
# Step 16: Choose final model
# Since the AUC values for all 3 - X_Train, X_Test1 and X_Test2 is more in the XGBoost model than the 
# NN model and the bias variance is also lesser, we choose XGBoost Model.

In [155]:
# Step 17: Strategy function
def calculate_strategy_performance(data, target_col, model_output_col, balance_col, spend_col, threshold):
    accepted = data[data[model_output_col] < threshold]
    rejected = data[data[model_output_col] >= threshold]
    
    portfolio_default_rate = accepted[target_col].mean()
    
    accepted_defaulters = accepted[accepted[target_col] == 1]
    accepted_non_defaulters = accepted[accepted[target_col] == 0]
    
    defaulter_revenue = 0
    non_defaulter_revenue = accepted_non_defaulters[balance_col].mean() * 0.02 + accepted_non_defaulters[spend_col].mean() * 0.001
    non_defaulter_revenue *= 12  # For 12 months
    
    portfolio_revenue = non_defaulter_revenue * len(accepted_non_defaulters)
    
    return portfolio_default_rate, portfolio_revenue